In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("Sample - Superstore.csv", encoding='latin1')


In [4]:
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


In [5]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [7]:
df.isna().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [12]:
df['Order Date']

0        11/8/2016
1        11/8/2016
2        6/12/2016
3       10/11/2015
4       10/11/2015
           ...    
9989     1/21/2014
9990     2/26/2017
9991     2/26/2017
9992     2/26/2017
9993      5/4/2017
Name: Order Date, Length: 9994, dtype: object

In [11]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [13]:
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [15]:
df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce')


In [18]:
df['Profit'] = pd.to_numeric(df['Profit'], errors='coerce')

In [19]:
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

In [ ]:
text_cols = ['Region', 'Category', 'Sub-Category', 'Product Name', 'Customer Name', 'City', 'State']
for c in text_cols:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()
        if c not in ['Product Name', 'Customer Name']:
            df[c] = df[c].str.title()

In [ ]:

df['Region'] = df['Region'].str.strip().str.title()
df['Category'] = df['Category'].str.strip().str.title()
df['Sub-Category'] = df['Sub-Category'].str.strip().str.title()


In [ ]:
#creating new columns for date analysis
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Month-Year'] = df['Order Date'].dt.to_period('M').astype(str)
df['MonthStart'] = df['Order Date'].dt.to_period('M').dt.to_timestamp()


In [ ]:
#importing visualization libraries
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets, HBox, VBox, Output

Sales Trend Over Time

In [27]:
monthly_sales = df.groupby('MonthStart', as_index=False)['Sales'].sum()
fig_trend = px.line(monthly_sales, x='MonthStart', y='Sales',
                    title="Sales Trend Over Time", markers=True)
fig_trend.update_xaxes(tickformat="%b %Y")
fig_trend.show()


Top 5 Products by Revenue

In [39]:
top5 = df.groupby('Product Name', as_index=False)['Sales'].sum().nlargest(5, 'Sales')

fig = px.bar(
    top5,
    x='Sales',
    y='Product Name',
    orientation='h',
    text='Sales',
    title="Top 5 Products by Revenue",
    color='Sales',
    color_continuous_scale='Blues'
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='inside')
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()


 Region-Wise Performance

In [40]:
region_perf = df.groupby('Region', as_index=False)['Sales'].sum()
fig_region = px.bar(region_perf, x='Region', y='Sales', color='Region',
                    title="Region-wise Sales Performance", text='Sales')
fig_region.show()


COMBINING ALL TOGETHER

In [41]:
out = Output()

regions = ['All'] + sorted(df['Region'].unique())
categories = ['All'] + sorted(df['Category'].unique())
years = ['All'] + sorted(df['Year'].unique().astype(str))

region_w = widgets.Dropdown(options=regions, description='Region:', value='All')
category_w = widgets.Dropdown(options=categories, description='Category:', value='All')
year_w = widgets.Dropdown(options=years, description='Year:', value='All')

def update(region, category, year):
    with out:
        out.clear_output()
        df_filtered = df.copy()
        
        # Apply filters
        if region != 'All':
            df_filtered = df_filtered[df_filtered['Region'] == region]
        if category != 'All':
            df_filtered = df_filtered[df_filtered['Category'] == category]
        if year != 'All':
            df_filtered = df_filtered[df_filtered['Year'] == int(year)]
        
        # 1️⃣ Sales trend
        monthly = df_filtered.groupby('MonthStart', as_index=False)['Sales'].sum()
        fig1 = px.line(monthly, x='MonthStart', y='Sales', 
                       title="Filtered Sales Trend", markers=True)
        fig1.show()

        # 2️⃣ Top 5 products - horizontal chart (improved readability)
        top5 = df_filtered.groupby('Product Name', as_index=False)['Sales'].sum().nlargest(5, 'Sales')
        fig2 = px.bar(
            top5,
            x='Sales',
            y='Product Name',
            orientation='h',
            text='Sales',
            title="Top 5 Products by Revenue",
            color='Sales',
            color_continuous_scale='Blues'
        )
        fig2.update_traces(texttemplate='%{text:.2f}', textposition='inside')
        fig2.update_layout(yaxis={'categoryorder': 'total ascending'})
        fig2.show()

        # 3️⃣ Region performance (always show all regions)
        all_regions_df = pd.DataFrame({'Region': sorted(df['Region'].unique())})
        region_perf = df_filtered.groupby('Region', as_index=False)['Sales'].sum()
        region_perf = all_regions_df.merge(region_perf, on='Region', how='left').fillna({'Sales': 0})

        fig3 = px.bar(region_perf, x='Region', y='Sales', color='Region', 
                      title="Region Performance", text='Sales')
        fig3.show()

# UI layout
ui = HBox([region_w, category_w, year_w])
display(VBox([ui, out]))

# Event bindings
region_w.observe(lambda change: update(region_w.value, category_w.value, year_w.value), names='value')
category_w.observe(lambda change: update(region_w.value, category_w.value, year_w.value), names='value')
year_w.observe(lambda change: update(region_w.value, category_w.value, year_w.value), names='value')

# Initial display
update(region_w.value, category_w.value, year_w.value)


In [32]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Prepare monthly sales data
monthly_sales = df.groupby('MonthStart', as_index=False)['Sales'].sum().sort_values('MonthStart')
monthly_sales['Month_Index'] = range(len(monthly_sales))  # time index

# Features (X) and target (y)
X = monthly_sales[['Month_Index']]
y = monthly_sales['Sales']

# Train the model
model = LinearRegression()
model.fit(X, y)

# Predict next month
next_month_index = monthly_sales['Month_Index'].max() + 1
predicted_sales = model.predict(np.array([[next_month_index]]))[0]

print(f"📈 Predicted Sales for Next Month: ${predicted_sales:,.2f}")

# Visualization of prediction
fig_forecast = px.line(monthly_sales, x='MonthStart', y='Sales', title="Sales Forecast", markers=True)
# Add predicted point
forecast_date = monthly_sales['MonthStart'].max() + pd.DateOffset(months=1)
fig_forecast.add_scatter(x=[forecast_date], y=[predicted_sales],
                         mode='markers+text', name='Forecast',
                         text=["Forecast"], textposition="top center",
                         marker=dict(color='red', size=10))
fig_forecast.show()


📈 Predicted Sales for Next Month: $69,957.54


c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



In [43]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Prepare monthly sales data
monthly_sales = df.groupby('MonthStart', as_index=False)['Sales'].sum().sort_values('MonthStart')

# Create time-based features
monthly_sales['Month'] = monthly_sales['MonthStart'].dt.month
monthly_sales['Year'] = monthly_sales['MonthStart'].dt.year
monthly_sales['Month_Index'] = range(len(monthly_sales))  # sequential index

# Features (X) and target (y)
X = monthly_sales[['Month_Index', 'Month', 'Year']]
y = monthly_sales['Sales']

# Train model
model = LinearRegression()
model.fit(X, y)

# Forecast next 6 months
future_months = 6
last_index = monthly_sales['Month_Index'].max()
future_dates = [monthly_sales['MonthStart'].max() + pd.DateOffset(months=i) for i in range(1, future_months+1)]

future_df = pd.DataFrame({
    'Month_Index': range(last_index+1, last_index+future_months+1),
    'Month': [d.month for d in future_dates],
    'Year': [d.year for d in future_dates]
})

predictions = model.predict(future_df)

# Visualization
fig_forecast = px.line(monthly_sales, x='MonthStart', y='Sales', title="Sales Forecast (Next 6 Months)", markers=True)

# Add forecast line & points
fig_forecast.add_scatter(
    x=future_dates, y=predictions,
    mode='lines+markers+text', name='Forecast',
    text=[f"{p:,.0f}" for p in predictions],
    textposition="top center",
    line=dict(color='red', dash='dash'),
    marker=dict(size=8, color='red')
)

fig_forecast.show()

# Print predictions
for date, pred in zip(future_dates, predictions):
    print(f"📅 {date.strftime('%b %Y')}: ${pred:,.2f}")


📅 Jan 2018: $37,501.75
📅 Feb 2018: $42,739.22
📅 Mar 2018: $47,976.70
📅 Apr 2018: $53,214.17
📅 May 2018: $58,451.65
📅 Jun 2018: $63,689.12
